In [1]:
import pyforest
import sys
sys.path.append('/Users/jayceepang/msse/ATOM_CODE/UTILS/')
from RF_functions import *
from GP_functions import *
from GenUtils import *
from dataset import *
import os
import pickle
from imblearn.over_sampling import SMOTEN, ADASYN, SMOTE
from sklearn.metrics import accuracy_score, balanced_accuracy_score,precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score, confusion_matrix,matthews_corrcoef

from rdkit import Chem
from rdkit.Chem import AllChem
import math
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import os

import shutil
import sklearn
from sklearn.model_selection import KFold
import gpytorch
from gpytorch.models import ExactGP
from gpytorch.likelihoods import DirichletClassificationLikelihood
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel, MaternKernel

from sklearn.metrics import confusion_matrix
import itertools
from sklearn.metrics import precision_score, recall_score, roc_auc_score, matthews_corrcoef, balanced_accuracy_score, confusion_matrix, f1_score, roc_curve,precision_recall_curve, auc
# from scipy.stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score, confusion_matrix

import sys

# from RF_GSCV import * # RF_GSCV contains the calculate metrics function to get the TP, TN, FP, FN scores 
# from RF_atomver import prediction_type 


In [2]:
og_neks = ['NEK2_binding','NEK2_inhibition','NEK3_binding','NEK5_binding','NEK9_binding','NEK9_inhibition']
samplings =['none_scaled','UNDER','SMOTE','ADASYN'] 
new_neks=['NEK1', 'NEK11', 'NEK2', 'NEK3', 'NEK4', 'NEK5',
       'NEK6', 'NEK9']
og_datapath = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/datasets/80train_20test/featurized/'
results_dir='/Users/jayceepang/msse/ATOM_CODE/examples/holdout/holdout_results/'
gp_results_dir='/Users/jayceepang/msse/ATOM_CODE/examples/holdout/holdout_results/GP/'
datapath = '/Users/jayceepang/msse/ATOM_CODE/datasets/holdout_data/featurized/'
gp_path ='/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/GP_results/'
test_results=[]
feat_cols_dropped =['NEK','compound_id','base_rdkit_smiles', 'subset', 'active'] 

In [3]:
def load_and_predict(model_path, likelihood_path, testX, testy, kernel='matern'):
    """
    Get pickled model+likelihood and run predictions new datasets

    @args:
        model_path (str): path topickled model file
        likelihood_path (str): Path to the pickled likelihood file.
        testX (torch.Tensor), testy (torch.Tensor)
        kernel (str): Kernel type RBF or matern
    @returns
        test_perf_df (pd.DataFrame): Prediction results and metrics on test data.
    """
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    with open(likelihood_path, 'rb') as f:
        likelihood = pickle.load(f)

    trainer = Trainer(model, likelihood, iterations=300)
    test_dist, test_observed_pred, test_pred_means, test_pred = trainer.predict(testX)
    test_results = trainer.gp_results(testX, testy)

    test_pred_samples = test_dist.sample(torch.Size((100,))).exp()

    test_probabilities = (test_pred_samples / test_pred_samples.sum(-2, keepdim=True)).mean(0)
    test_prob_stds = (test_pred_samples / test_pred_samples.sum(-2, keepdim=True)).std(0)

    test_perf_df = pd.DataFrame()
    test_perf_df['mean_pred_class0'] = test_observed_pred.mean.numpy()[0,]
    test_perf_df['mean_pred_class1'] = test_observed_pred.mean.numpy()[1,]
    test_perf_df['y'] = testy
    test_perf_df['y_pred'] = test_pred_means.max(0)[1]
    test_perf_df['var_pred_class0'] = test_observed_pred.variance.numpy()[0,]
    test_perf_df['var_pred_class1'] = test_observed_pred.variance.numpy()[1,]
    test_perf_df['pred_prob_class0'] = test_probabilities.numpy()[0,]
    test_perf_df['pred_prob_class1'] = test_probabilities.numpy()[1,]
    test_perf_df['pred_prob_std_class0'] = test_prob_stds.numpy()[0,]
    test_perf_df['pred_prob_std_class1'] = test_prob_stds.numpy()[1,]
    test_perf_df['subset'] = 'nanobret_cellular_holdout'

    for k, val in test_results.items():
        test_perf_df[k] = val

    return test_perf_df


In [4]:
# test_results=[]
# final_cols=['model','NEK','strategy','feat_type', 'cm','recall', 'specificity', 'accuracy', 'precision', 
#                 'f1', 'ROC_AUC', 'MCC', 'balanced_accuracy', 'prod_model']
# for nek in new_neks: 
#     for i in og_neks: 
#         for samp in samplings: 
#             trainX=pd.read_csv(f'{og_datapath}{i}_MFP_{samp}_trainX.csv')
#             train_y =pd.read_csv(f'{og_datapath}{i}_MFP_{samp}_train_y.csv').to_numpy().reshape(-1)
#             testX=pd.read_csv(f'{datapath}{nek}_MFP_none_unscaled.csv').drop(columns=feat_cols_dropped)
#             test_y=pd.read_csv(f'{datapath}{nek}_MFP_none_unscaled.csv')['active'].to_numpy().reshape(-1)
#             model_name = f'{nek}_MFP_none_scaled'
#             # _,_, testX,testy=make_torch_tens_float(filepath=None, filename=None, rootname=None, df=None)
#             testX, testy = make_torch_tens_float_simple(file_path_X=None,file_path_y=None, df_path=f'{datapath}{nek}_MFP_none_unscaled.csv')
#             gp_model = f'{gp_path}{i}_MFP_{samp}_matern.pkl'
     
#             gp_likelihood = f'{gp_path}{i}_MFP_{samp}_matern_likelihood.pkl'
#             test_df = load_and_predict(gp_model,gp_likelihood, testX,testy, kernel='matern')
#             test_df['model'] = f'{model_name}_{i}_{samp}_GP_matern'
#             test_df['NEK'] =nek
            
#             test_df['feat_type'] = 'MFP'
#             test_df['strategy'] = samp
#             test_df['kernel_type'] = 'GP_matern'
#             test_df['prod_model'] = f'{i}_{samp}_GP_matern'
            
            
#             test_results.append(test_df.iloc[[0]][final_cols].values.flatten())
            
#             test_df.to_csv(f'{gp_results_dir}{model_name}_{i}_{samp}_GP_matern_nanobret_cellular_holdout.csv', index=False)

            
#     all_test =  pd.DataFrame(test_results,columns=final_cols)
#     all_test['modeling_type'] = 'GP' 
#     all_test['set'] = 'nanobret_cellular_holdout' 

#     # all_test.to_csv(f'{gp_results_dir}nanobret_cellular_holdout_results_all_NEK.csv', index=False)                 
         

In [5]:
test_results=[]
final_cols=['model','prod_model','NEK','data NEK', 'model NEK','strategy','feat_type', 'cm',
            'recall', 'specificity', 'accuracy', 'precision', 
                'f1', 'ROC_AUC', 'MCC', 'balanced_accuracy', '1-specificity', 'model strategy']
for nek in new_neks:
    testX=pd.read_csv(f'{datapath}{nek}_MFP_none_unscaled.csv').drop(columns=feat_cols_dropped)
    test_y=pd.read_csv(f'{datapath}{nek}_MFP_none_unscaled.csv')['active'].to_numpy().reshape(-1)
    for i in og_neks: 
        for samp in samplings: 
            trainX=pd.read_csv(f'{og_datapath}{i}_MFP_{samp}_trainX.csv')
            train_y =pd.read_csv(f'{og_datapath}{i}_MFP_{samp}_train_y.csv').to_numpy().reshape(-1)
            
            model_name = f'{nek}_MFP_none_scaled'
            print(nek, model_name) 
            # _,_, testX,testy=make_torch_tens_float(filepath=None, filename=None, rootname=None, df=None)
            testX, testy = make_torch_tens_float_simple(file_path_X=None,file_path_y=None, df_path=f'{datapath}{nek}_MFP_none_unscaled.csv')
            gp_model = f'{gp_path}{i}_MFP_{samp}_matern.pkl'
     
            gp_likelihood = f'{gp_path}{i}_MFP_{samp}_matern_likelihood.pkl'
            test_df = load_and_predict(gp_model,gp_likelihood, testX,testy, kernel='matern')
            test_df['model'] = f'{nek}_{i}_MFP_{samp}_GP_matern' # UNDER_GP_matern
            test_df['prod_model'] = f'{i}_{samp}_GP_matern'
            test_df['data NEK'] =nek
            test_df['model NEK'] = i
            test_df['NEK']=nek
            test_df['feat_type'] = 'MFP'
            test_df['strategy'] = samp
            # test_df['kernel_type'] = 'GP_matern' 
            test_df['model strategy'] = f'{samp}_GP_matern'
            test_df['1-specificity']=1-test_df['specificity'] 
            test_df.to_csv(f'{gp_results_dir}{model_name}_{i}_{samp}_GP_matern_nanobret_cellular_holdout.csv', index=False)
            test_results.append(test_df.iloc[[0]][final_cols].values.flatten())
            
    all_test =  pd.DataFrame(test_results,columns=final_cols)
    all_test['modeling_type'] = 'GP' 
    all_test['set'] = 'nanobret_cellular_holdout' 

    all_test.to_csv(f'{gp_results_dir}GP_nanobret_cellular_holdout_results_all_NEK.csv', index=False)                 
         

NEK1 NEK1_MFP_none_scaled


/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packa

accuracy: 0.9250, precision: 0.0000, recall: 0.0000, specificity: 1.0000, cm: [[370   0]
 [ 30   0]]
NEK1 NEK1_MFP_none_scaled
accuracy: 0.4550, precision: 0.1148, recall: 0.9333, specificity: 0.4162, cm: [[154 216]
 [  2  28]]
NEK1 NEK1_MFP_none_scaled


/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(


accuracy: 0.9250, precision: 0.0000, recall: 0.0000, specificity: 1.0000, cm: [[370   0]
 [ 30   0]]
NEK1 NEK1_MFP_none_scaled


...

/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavio

accuracy: 0.9150, precision: 0.0000, recall: 0.0000, specificity: 1.0000, cm: [[366   0]
 [ 34   0]]
NEK9 NEK9_MFP_none_scaled
accuracy: 0.9150, precision: 0.0000, recall: 0.0000, specificity: 1.0000, cm: [[366   0]
 [ 34   0]]
NEK9 NEK9_MFP_none_scaled
accuracy: 0.9125, precision: 0.3333, recall: 0.0294, specificity: 0.9945, cm: [[364   2]
 [ 33   1]]
NEK9 NEK9_MFP_none_scaled
accuracy: 0.9150, precision: 0.0000, recall: 0.0000, specificity: 1.0000, cm: [[366   0]
 [ 34   0]]


/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-04 to the diagonal
  warnings.warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/linear_operator/operators/_linear_operator.py:2163: NumericalWarning: Runtime Error when computing Cholesky decomposition: Matrix not positive definite after repeatedly adding jitter up to 1.0e-04.. Using symeig method.
  warnings.warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/sklearn/metrics/_classificat

NEK9 NEK9_MFP_none_scaled
accuracy: 0.9150, precision: 0.0000, recall: 0.0000, specificity: 1.0000, cm: [[366   0]
 [ 34   0]]


/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/Users/jayceepang/miniconda3/envs/ATOM_CODE/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(


In [6]:
# all_test = pd.read_csv(f'{gp_results_dir}GP_nanobret_cellular_holdout_results.csv')
all_test.columns

Index(['model', 'prod_model', 'NEK', 'data NEK', 'model NEK', 'strategy',
       'feat_type', 'cm', 'recall', 'specificity', 'accuracy', 'precision',
       'f1', 'ROC_AUC', 'MCC', 'balanced_accuracy', '1-specificity',
       'model strategy', 'modeling_type', 'set'],
      dtype='object')

In [7]:
# all_test['data NEK'] = all_test['NEK'] 
all_test['model NEK'] = all_test['prod_model'].astype(str).str.split('_').str[0:2].apply(lambda x: '_'.join(x))
all_test=all_test.rename(columns={'NEK':'data NEK','strategy': 'model strategy'})
all_test['model version'] = 'GP_matern'
new_order = ['model', 'data NEK', 'prod_model', 'model NEK', 'model version','model strategy', 'feat_type', 'cm', 'recall',
             'specificity', 'accuracy', 'precision', 'f1', 'ROC_AUC', 'MCC', 'balanced_accuracy',
             'set', 'modeling_type']
all_test = all_test[new_order]
# all_test.to_csv(f'{gp_results_dir}GP_nanobret_cellular_holdout_results.csv', index=False)

In [8]:
all_test

,model,data NEK,data NEK,prod_model,model NEK,model version,model strategy,model strategy,feat_type,cm,recall,specificity,accuracy,precision,f1,ROC_AUC,MCC,balanced_accuracy,set,modeling_type
0,NEK1_NEK2_binding_MFP_none_scaled_GP_matern,NEK1,NEK1,NEK2_binding_none_scaled_GP_matern,NEK2_binding,GP_matern,none_scaled,none_scaled_GP_matern,MFP,"[370, 0, 30, 0]",0.000000,1.000000,0.9250,0.000000,0.000000,0.500000,0.000000,0.500000,nanobret_cellular_holdout,GP
1,NEK1_NEK2_binding_MFP_UNDER_GP_matern,NEK1,NEK1,NEK2_binding_UNDER_GP_matern,NEK2_binding,GP_matern,UNDER,UNDER_GP_matern,MFP,"[154, 216, 2, 28]",0.933333,0.416216,0.4550,0.114754,0.204380,0.674775,0.188761,0.674775,nanobret_cellular_holdout,GP
2,NEK1_NEK2_binding_MFP_SMOTE_GP_matern,NEK1,NEK1,NEK2_binding_SMOTE_GP_matern,NEK2_binding,GP_matern,SMOTE,SMOTE_GP_matern,MFP,"[370, 0, 30, 0]",0.000000,1.000000,0.9250,0.000000,0.000000,0.500000,0.000000,0.500000,nanobret_cellular_holdout,GP
3,NEK1_NEK2_binding_MFP_ADASYN_GP_matern,NEK1,NEK1,NEK2_binding_ADASYN_GP_matern,NEK2_binding,GP_matern,ADASYN,ADASYN_GP_matern,MFP,"[370, 0, 30, 0]",0.000000,1.000000,0.9250,0.000000,0.000000,0.500000,0.000000,0.500000,nanobret_cellular_holdout,GP
4,NEK1_NEK2_inhibition_MFP_none_scaled_GP_matern,NEK1,NEK1,NEK2_inhibition_none_scaled_GP_matern,NEK2_inhibition,GP_matern,none_scaled,none_scaled_GP_matern,MFP,"[370, 0, 30, 0]",0.000000,1.000000,0.9250,0.000000,0.000000,0.500000,0.000000,0.500000,nanobret_cellular_holdout,GP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,NEK9_NEK9_binding_MFP_ADASYN_GP_matern,NEK9,NEK9,NEK9_binding_ADASYN_GP_matern,NEK9_binding,GP_matern,ADASYN,ADASYN_GP_matern,MFP,"[366, 0, 34, 0]",0.000000,1.000000,0.9150,0.000000,0.000000,0.500000,0.000000,0.500000,nanobret_cellular_holdout,GP
188,NEK9_NEK9_inhibition_MFP_none_scaled_GP_matern,NEK9,NEK9,NEK9_inhibition_none_scaled_GP_matern,NEK9_inhibition,GP_matern,none_scaled,none_scaled_GP_matern,MFP,"[366, 0, 34, 0]",0.000000,1.000000,0.9150,0.000000,0.000000,0.500000,0.000000,0.500000,nanobret_cellular_holdout,GP
189,NEK9_NEK9_inhibition_MFP_UNDER_GP_matern,NEK9,NEK9,NEK9_inhibition_UNDER_GP_matern,NEK9_inhibition,GP_matern,UNDER,UNDER_GP_matern,MFP,"[364, 2, 33, 1]",0.029412,0.994536,0.9125,0.333333,0.054054,0.511974,0.077407,0.511974,nanobret_cellular_holdout,GP
190,NEK9_NEK9_inhibition_MFP_SMOTE_GP_matern,NEK9,NEK9,NEK9_inhibition_SMOTE_GP_matern,NEK9_inhibition,GP_matern,SMOTE,SMOTE_GP_matern,MFP,"[366, 0, 34, 0]",0.000000,1.000000,0.9150,0.000000,0.000000,0.500000,0.000000,0.500000,nanobret_cellular_holdout,GP
